# Design Notes

## Main goal of pipeline maker

* We want the user to specify a **collection of objects**
* more concretely: 
  * connect a **datasource** to an existing dacc, in a normalized form (say, a dacc using WfStore, AnnotStore)
  * datasource can be anything, but the (raw) values we want are numpy arrays (int16). 
  The keys can be deduced from the items, after a sequence of transformations (see mk_item2kv_for, and other functions in dol for example). 
  Another tool to use : dol/filt_iter on store
  We can use the interface of dol/Files to get the raw material (example: FileBytesReader)

* At the **GUI level**: user makes a collection by adding funcs
  * 'select one' : from pre-existing dropdown list
  * 'make one' and select after (will use crude: we select through the crude name)

* At the **backend level**:
  * two stores are provided: one store of funcs (populated with 2-3 funcs say), one store of factories.
  * make a collection of functions through the GUI (in the form of SlabsIter or not)
  * the user needs to populate the items. Behind the scenes we provide functions to create the list of objects that we can select. Those functions can be provided as keys, of a mall. If very large, the collection of keys is accessed via a smartlist. The function that creates the items could be as simple as dunder iter for the store.
  * making one element: we provide a store of factories. Those have certain signatures, that will determine the front rendering of each. For example: one factory needs some params to be entered by the user, so front creates a page showing the arguments of the factory. User enters arguments, then click to save in (dill) store of functions.
  * In summary: there are 2 stores, store of funcs, store of factories

* **Remark**: crude:
  does not appear at the level of the React components. 

* **example**:
    make a pipeline to create a SlabsIter example (ie a store exists, and is transformed by adding more stuff to it)
  

# A SlabsIter example

* random generator: provided directly
* clipper: clip above fixed value
* threshold maker: a factory
(* jump finder: a factory (define jump))


In [39]:
import numpy as np
from functools import partial
from know.base import SlabsIter


DFLT_SIZE = 5
DFLT_WIDTH = 10

def random_gen(width= DFLT_WIDTH, size = DFLT_SIZE):
    while True:
        yield np.random.randint(width, size=size)

def clipper(arr, a_min=0, a_max=10):
    return np.clip(arr, a_min, a_max)

def threshold_factory():
    max_val = int(input())
    return partial(clipper, a_min=0, a_max = max_val)


data = random_gen().__next__
clipper_4 = partial(clipper, a_min=0, a_max = 4)

funcs_store = {'clipper': clipper_4}
factories_store = {'threshold': threshold_factory}

transform = funcs_store['clipper']


In [40]:
s = SlabsIter(
    data = data, 
    transform = lambda data: data>3
)

In [41]:
next(s)

{'data': array([0, 8, 3, 4, 5]),
 'transform': array([False,  True, False,  True,  True])}

In [42]:
data()

array([0, 6, 4, 0, 6])

In [43]:
data()

array([0, 8, 3, 7, 8])